<a href="https://colab.research.google.com/github/MariaOmaechevarria/OPENCL/blob/main/FILTROS%20IMAGENES/Filtro_color_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt purge *nvidia* -y
!sudo apt install nvidia-driver-530 -y


!pip install pyopencl
!apt-get install -y pocl-opencl-icd ocl-icd-libopencl1

In [ ]:
import pyopencl as cl
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

from PIL import Image, ImageFilter

In [ ]:
kernel_filter_color_local="""
__kernel void kernel_filter_color_local(__global uchar* imagen_in, __global uchar* imagen_out, __constant float* filtro, int dim, int ancho, int alto, __local uchar* local_imagen) {

    int fila = get_global_id(0);
    int columna = get_global_id(1);

    int centro = (dim - 1) / 2;

    // IDs locales
    int local_fila = get_local_id(0);
    int local_columna = get_local_id(1);

    // Tamaño del grupo de trabajo
    int local_size_x = get_local_size(0);
    int local_size_y = get_local_size(1);

    // Índice para la memoria local (considerando 3 canales de color)
    int local_idx = (local_fila * (local_size_x + 2 * centro) + local_columna) * 3;

    // Índice para la memoria global
    int global_idx = (fila * ancho + columna) * 3;

    // Cargar los píxeles desde la memoria global a la memoria local
    for (int c = 0; c < 3; c++) {  // Iterar sobre los canales de color (R, G, B)
        if (fila < alto && columna < ancho) {
            local_imagen[local_idx + c] = imagen_in[global_idx + c];
        }

        // Cargar bordes si es necesario
        if (local_fila < centro && fila >= centro) {
            local_imagen[local_idx + c - centro * (local_size_x + 2 * centro) * 3] = imagen_in[global_idx + c - centro * ancho * 3];
        }
        if (local_columna < centro && columna >= centro) {
            local_imagen[local_idx + c - centro * 3] = imagen_in[global_idx + c - centro * 3];
        }
    }

    // Sincronizar para asegurarse de que todos los datos están cargados
    barrier(CLK_LOCAL_MEM_FENCE);

    // Variables de acumulación
    float suma_rojo = 0.0f;
    float suma_verde = 0.0f;
    float suma_azul = 0.0f;

    // Aplicar el filtro usando los valores de la memoria local
    if (centro <= fila && fila < (alto - centro) && centro <= columna && columna < (ancho - centro)) {
        for (int i = -centro; i <= centro; i++) {
            for (int j = -centro; j <= centro; j++) {
                int local_idx_nuevo = ((local_fila + i + centro) * (local_size_x + 2 * centro) + (local_columna + j + centro)) * 3;

                float pixel_rojo = local_imagen[local_idx_nuevo + 0];
                float pixel_verde = local_imagen[local_idx_nuevo + 1];
                float pixel_azul = local_imagen[local_idx_nuevo + 2];

                float valor_filtro = filtro[(i + centro) * dim + (j + centro)];

                suma_rojo += pixel_rojo * valor_filtro;
                suma_verde += pixel_verde * valor_filtro;
                suma_azul += pixel_azul * valor_filtro;
            }
        }

        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = (uchar)suma_rojo;
        imagen_out[idx_out + 1] = (uchar)suma_verde;
        imagen_out[idx_out + 2] = (uchar)suma_azul;
    } else {
        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = imagen_in[idx_out];
        imagen_out[idx_out + 1] = imagen_in[idx_out + 1];
        imagen_out[idx_out + 2] = imagen_in[idx_out + 2];
    }
}


"""

In [ ]:
def importar_imagen(path):
  imagen = Image.open(path)
  return imagen

In [ ]:
def aplicar_filtro_color_local(image_path, filtro, kernel_code, kernel_name, device_type, local_size):
    # Importar imagen
    imagen = importar_imagen(image_path)

    # Convertirla a un array de tres canales
    imagen_np = np.array(imagen).astype(np.uint8)

    # Dimensiones de la imagen
    tam_x, tam_y, _ = imagen_np.shape
    print(tam_x,tam_y)

    # Crear array para la imagen final
    imagen_out_np = np.empty_like(imagen_np)

    # Plataforma y dispositivo
    platform = cl.get_platforms()[0]
    device = platform.get_devices(device_type=device_type)[0]

    # Crear contexto y cola de comandos
    context = cl.Context([device])
    command_queue = cl.CommandQueue(context, device=device, properties=cl.command_queue_properties.PROFILING_ENABLE)

    # Crear el programa y compilarlo
    program = cl.Program(context, kernel_code).build()

    # Crear el kernel
    kernel_filter = cl.Kernel(program, kernel_name)

    # Obtener el tamaño máximo del grupo de trabajo
    wg_size = kernel_filter.get_work_group_info(cl.kernel_work_group_info.WORK_GROUP_SIZE, device)
    print(f"Tamaño máximo del grupo de trabajo: {wg_size}")

    # Crear buffers de imagen
    buffer_in = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=imagen_np)
    buffer_out = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, imagen_out_np.nbytes)

    # Crear buffer para el filtro
    filtro_buf = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=filtro)

    #Asignar memoria local
    dim=filtro.shape[0]
    centro = (dim - 1) // 2
    local_size_x, local_size_y = local_size
    local_mem_size = (local_size_x + 2 * centro) * (local_size_y + 2 * centro) * 3  # 3 es por los canales R, G, B

    # Define la memoria local
    local_mem = cl.LocalMemory(local_mem_size)

    # Argumentos del kernel
    kernel_filter.set_arg(0, buffer_in)
    kernel_filter.set_arg(1, buffer_out)
    kernel_filter.set_arg(2, filtro_buf)
    kernel_filter.set_arg(3, np.int32(filtro.shape[0]))
    kernel_filter.set_arg(4, np.int32(imagen_np.shape[1]))
    kernel_filter.set_arg(5, np.int32(imagen_np.shape[0]))
    kernel_filter.set_arg(6, local_mem)

    # Ejecutar el kernel
    global_size = (tam_x, tam_y)
    event = cl.enqueue_nd_range_kernel(command_queue, kernel_filter, global_size, local_size)
    event.wait()

    # Leer el buffer de salida
    cl.enqueue_copy(command_queue, imagen_out_np, buffer_out)

    # Obtener el tiempo de ejecución
    exec_time = 1e-9 * (event.profile.end - event.profile.start)

    # Guardar y mostrar la imagen resultante
    imagen_resultante = Image.fromarray(imagen_out_np)
    imagen_resultante.show()
    #imagen_resultante.save("/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/pirineos_filtro_color_local.png")

    return imagen_resultante, exec_time


In [ ]:
#SUAVIZAR LA IMAGEN
filtro_mean=np.array([
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9]
], dtype=np.float32)

#SUAVIZAR IMAGEN,ELIMINA LOS VALORES DE ALTAS FRECUENCIAS
filtro_gaussiani=np.array([
    [1/16, 2/16, 1/16],
    [2/16, 4/16, 2/16],
    [1/16, 2/16, 1/16]
], dtype=np.float32)

#ENFOCA
filtro_enfoque=np.array([
    [0, -1, 0],
    [-1, 5, 1],
    [0, -1, 0]
], dtype=np.float32)

#DESENFOCA
filtro_desenfoque=np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
], dtype=np.float32)

#FILTRO SOBEL: DETECTA BORDES

filtro_sobel_X=np.array([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
], dtype=np.float32)

filtro_sobel_Y=np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)


In [ ]:

def filtro_medio_local():
    device_type = cl.device_type.CPU
    image_path = "/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/IMAGENES/pirineos_imagen.png"

    # Definir las combinaciones de local_size
    local_sizes = [(1, 1), (2, 2), (4, 4), (8, 8), (16, 16), (16, 8), (8, 16)]

    # Crear columnas para el DataFrame
    columns = [f"({ls[0]}/{ls[1]})" for ls in local_sizes]
    results_df = pd.DataFrame(index=['Imagen Pirineos'], columns=columns)

    # Probar cada local_size y medir el tiempo de ejecución
    for local_size in local_sizes:
        imagen_resultante, exec_time = aplicar_filtro_color_local(image_path, filtro_mean, kernel_filter_color_local, "kernel_filter_color_local", device_type, local_size)

        # Asignar el tiempo de ejecución al DataFrame
        results_df.loc['Imagen Pirineos', f"({local_size[0]}/{local_size[1]})"] = exec_time if exec_time is not None else "NP"
        print(f"local_size: {local_size}, tiempo de ejecución: {exec_time}")

    # Guardar los resultados en un archivo CSV
    results_df.to_csv('/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/Filtros_mean_local_CPU.csv', index=True)

filtro_medio_local()